# Take the Distance Data and Compute an MST

In [128]:
import pandas as pd 
from networkx import *

distance_df = pd.read_csv("fridge_distance.csv")
g = Graph()
for index,row in distance_df.iterrows():
    g.add_edge(row['LocationA'],row['LocationB'],weight=row['Straight Line Distance (mi.)'])

mst = list(minimum_spanning_edges(g))
mst

[('Hull Street', 'Fonticello', {'weight': 0.2}),
 ('Highland Park', 'Six Points Innovation Center', {'weight': 0.27}),
 ('Matchbox Mutual Aid', 'Six Points Innovation Center', {'weight': 0.79}),
 ('Six Points Innovation Center',
  'Meadowbridge Community Market',
  {'weight': 0.79}),
 ('Church Hill', 'Chimborazo', {'weight': 0.96}),
 ('Fulton', 'Chimborazo', {'weight': 1.13}),
 ('Dr. John Jones Office', 'Carytown', {'weight': 1.18}),
 ('Fonticello', 'South Side', {'weight': 1.93}),
 ('Dr. John Jones Office', 'Scotts Addition', {'weight': 1.94}),
 ('Dr. John Jones Office', 'Church Hill', {'weight': 1.98}),
 ('Dr. John Jones Office', 'Hull Street', {'weight': 2.14}),
 ('Dr. John Jones Office', 'Matchbox Mutual Aid', {'weight': 2.16}),
 ('Fulton', 'East End', {'weight': 2.97}),
 ('South Side', 'VSU', {'weight': 18.98})]

# Take the MST and draw it on a map
## First convert all the addresses to coordinates

In [129]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

tqdm.pandas()
geolocator = Nominatim(user_agent="fridge-project")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_location(address):
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)

fridge_list = pd.read_csv("all_fridges.csv")
fridge_list = fridge_list[["Location Name", "Address"]]
fridge_list["Geocode"] = fridge_list["Address"].progress_apply(geocode)
fridge_list["Lat"] = fridge_list["Geocode"].apply(lambda loc: loc.point.latitude if loc else [None, None])
fridge_list["Long"] = fridge_list["Geocode"].apply(lambda loc: loc.point.longitude if loc else [None, None])
no_location = fridge_list[fridge_list["Geocode"].isnull()]
if no_location.shape[0] > 0:
    print(no_location.head())

100%|██████████| 16/16 [00:16<00:00,  1.03s/it]


## Create a map centered around VCU

In [126]:
import folium

midpoint = fridge_list[fridge_list["Location Name"] == "Dr. John Jones Office"]
midpoint = (midpoint["Lat"], midpoint["Long"])
m = folium.Map(location=midpoint, zoom_start=12)
# add markers for all the fridges
fridge_list.apply(lambda row: folium.Marker((row["Lat"], row["Long"]), popup=row["Location Name"]).add_to(m), axis=1)

def connect_locations(locA, locB): # take two location names and plot the line between them on the map
    coords_A = tuple(fridge_list[fridge_list["Location Name"] == locA][["Lat", "Long"]].iloc[0])
    coords_B = tuple(fridge_list[fridge_list["Location Name"] == locB][["Lat", "Long"]].iloc[0])
    folium.PolyLine([coords_A, coords_B], color="blue", weight=2.5, opacity=1).add_to(m)


for edge in mst:
    connect_locations(edge[0], edge[1])

m

/usr/local/lib/python3.11/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/usr/local/lib/python3.11/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/usr/local/lib/python3.11/site-packages/folium/utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
